In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import absolute_import, division, print_function

import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import skimage

from src.data_utils import Dataset
from src.network import unet_3d

tf.enable_eager_execution()

## Setup
- load train and test datasets, create TensorFlow Datasets from them
- define main architecutre params

In [2]:
train_dataset = Dataset.load_dataset(
    '../data/processed/train_dataset.pckl'
).create_tf_dataset().shuffle(50).repeat().batch(1)
test_dataset = Dataset.load_dataset(
    '../data/processed/test_dataset.pckl'
).create_tf_dataset().shuffle(50).repeat().batch(1)

In [ ]:
%load_ext autoreload
%autoreload 2

from __future__ import absolute_import, division, print_function

import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import skimage

from src.data_utils import Dataset
from src.network import unet_3d

# tf.enable_eager_execution()

train_dataset = Dataset.load_dataset(
    '../data/processed/train_dataset.pckl'
).create_tf_dataset().shuffle(50).repeat().batch(1)
test_dataset = Dataset.load_dataset(
    '../data/processed/test_dataset.pckl'
).create_tf_dataset().shuffle(50).repeat().batch(1)

iterator = tf.data.Iterator.from_structure(
    train_dataset.output_types,
    train_dataset.output_shapes
)
training_init_op = iterator.make_initializer(train_dataset)
test_init_op = iterator.make_initializer(test_dataset)
next_element = iterator.get_next()

In [5]:
avg_acc

array([7.0381794])

In [ ]:
# create the neural network model
logits = unet_3d(next_element[0], depth=3)

# add the optimizer and weigthed softmax loss
# see https://stackoverflow.com/a/44563055
class_weights = tf.cast(tf.constant([[0.00408978, 0.70013423, 0.295776]]), tf.float32)
# deduce weights for batch samples based on their true label
class_weights = tf.reduce_sum(tf.cast(next_element[0], tf.float32) * class_weights, axis=-1)
loss = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=next_element[1], weights=class_weights)
optimizer = tf.train.AdamOptimizer().minimize(loss)

# get accuracy
prediction = tf.argmax(logits, axis=-1)
labels = tf.argmax(next_element[1], -1)
equality = tf.equal(prediction, labels)
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

# get mean iou


# run the training
epochs = 20
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(training_init_op)
    for i in range(epochs):
        iou, conf_mat = tf.metrics.mean_iou(labels, tf.cast(prediction, tf.int32), 3)
        sess.run(tf.local_variables_initializer())
        l, _, acc, miou = sess.run([loss, optimizer, accuracy, iou])
        if i % 10 == 0:
            print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100))
            print("Epoch: {}, loss: {:.3f}, training IOU: {:.2f}%".format(i, l, miou * 100))
    # now setup the validation run
    valid_iters = 10
    # re-initialize the iterator, but this time with validation data
    sess.run(test_init_op)
    avg_acc = 0
    for i in range(valid_iters):
        acc = sess.run([accuracy])
        avg_acc += acc[0]
    print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters, (avg_acc / valid_iters) * 100))

conv1: (?, ?, 128, 128, 16)
conv2: (?, ?, 128, 128, 32)
maxpool layer: (?, ?, 64, 64, 32)
conv1: (?, ?, 64, 64, 32)
conv2: (?, ?, 64, 64, 64)
maxpool layer: (?, ?, 32, 32, 64)
conv1: (?, ?, 32, 32, 64)
conv2: (?, ?, 32, 32, 128)
upconv layer: (?, ?, 64, 64, 128)
concat layer: (?, ?, 64, 64, 64)
up_conv layer1: (?, ?, 64, 64, 64)
up_conv layer2: (?, ?, 64, 64, 64)
upconv layer: (?, ?, 128, 128, 64)
concat layer: (?, ?, 128, 128, 32)
up_conv layer1: (?, ?, 128, 128, 32)
up_conv layer2: (?, ?, 128, 128, 32)
output layer: (?, ?, 128, 128, 3)
Epoch: 0, loss: 0.168, training accuracy: 42.98%
Epoch: 0, loss: 0.168, training IOU: 0.00%
